In [1]:
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Markdown, display
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# to display all rows columns 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 1)

In [2]:
dfMatch1 = pd.read_csv(r"ind_sl_t20i_1st_2022_02_24.csv")
dfMatch1 = dfMatch1.join(pd.get_dummies(dfMatch1[['Role', 'Team']]))
dfMatch1.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,False,False,False,True,True,False
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,False,True,False,False,True,False
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,False,False,True,False,True,False
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,False,True,False,False,False,True
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,False,True,False,False,True,False


In [3]:
# function to calculate and create fantasy team
def createFanstasyTeam(df):

    df['PlayingXI'] = 1

    teams = list(df.Team.unique())
    team1, team2 = teams[0], teams[1]

    # define decision variable
    pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary')

    # define objective function
    mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Sel'] for obj in df.index])

    # define constraints
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

    # 11 players
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

    # team constraints
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team1}"]] for obj in df.index) >= 4
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team1}"]] for obj in df.index) <= 7

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team2}"]] for obj in df.index) >= 4
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team2}"]] for obj in df.index) <= 7

    # role constraints
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
    mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

    mo.solve()

    # output as a column in df
    df['pickup_status'] = 0

    for obj in df.index:
        if pickup_status[obj].varValue:
            df['pickup_status'][obj] = 1

    fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)
    return fdf

In [4]:
fdf1 = createFanstasyTeam(dfMatch1)

In [5]:
fdf1[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status
0,S Iyer,9428,IND,BAT,71.23,8.5,0,1,1
1,C Asalanka,10934,SL,BAT,64.60,8.5,0,1,1
2,R Sharma,576,IND,BAT,87.30,10.5,0,1,1
3,P Nissanka,13682,SL,BAT,85.62,9.5,0,1,1
4,I Kishan,10276,IND,WK,73.23,8.5,0,1,1
5,K Mendis,9494,SL,WK,53.58,9.0,0,1,1
6,R Jadeja,587,IND,AR,74.73,9.0,0,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,1
8,D Chameera,8393,SL,BOWL,78.22,9.0,0,1,1
9,H Patel,8175,IND,BOWL,65.20,8.5,0,1,1


In [6]:
fdf1[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'Points', 'DT', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,pickup_status
0,S Iyer,9428,IND,BAT,71.23,8.5,0,1,84,1,1
1,C Asalanka,10934,SL,BAT,64.60,8.5,0,1,70,1,1
2,R Sharma,576,IND,BAT,87.30,10.5,0,1,66,1,1
3,P Nissanka,13682,SL,BAT,85.62,9.5,0,1,2,0,1
4,I Kishan,10276,IND,WK,73.23,8.5,0,1,141,1,1
5,K Mendis,9494,SL,WK,53.58,9.0,0,1,0,0,1
6,R Jadeja,587,IND,AR,74.73,9.0,0,1,34,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,0,0,1
8,D Chameera,8393,SL,BOWL,78.22,9.0,0,1,30,1,1
9,H Patel,8175,IND,BOWL,65.20,8.5,0,1,8,0,1


In [7]:
match_1_points = sum(fdf1.Points)
printmd('**Total Points for Match 1: '+ str(match_1_points)+'**', 'blue')

<span style='color:blue'>**Total Points for Match 1: 441**</span>

In [9]:
dfMatch2 = pd.read_csv("ind_sl_t20i_2nd_2022_02_26.csv")
dfMatch2 = dfMatch2.join(pd.get_dummies(dfMatch2[['Role', 'Team']]))
dfMatch2.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,False,True,False,False,True,False
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,False,True,False,False,False,True
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,True,False,False,False,True,False
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,True,False,False,False,False,True
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,False,False,False,True,True,False


In [10]:
fdf2 = createFanstasyTeam(dfMatch2)

In [11]:
fdf2[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status
0,S Iyer,9428,IND,BAT,87.64,8.5,84,1,1
1,P Nissanka,13682,SL,BAT,66.77,9.5,2,1,1
2,R Sharma,576,IND,BAT,87.27,10.5,66,1,1
3,C Asalanka,10934,SL,BAT,77.97,8.5,70,1,1
4,I Kishan,10276,IND,WK,83.71,8.5,141,1,1
5,R Jadeja,587,IND,AR,76.24,9.0,34,1,1
6,D Shanaka,8422,SL,AR,68.26,9.0,32,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,1
8,B Kumar,1726,IND,BOWL,63.78,8.5,76,1,1
9,D Chameera,8393,SL,BOWL,72.85,9.0,30,1,1


In [12]:
fdf2[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'Points', 'DT', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,pickup_status
0,S Iyer,9428,IND,BAT,87.64,8.5,84,1,112,1,1
1,P Nissanka,13682,SL,BAT,66.77,9.5,2,1,100,1,1
2,R Sharma,576,IND,BAT,87.27,10.5,66,1,13,0,1
3,C Asalanka,10934,SL,BAT,77.97,8.5,70,1,6,0,1
4,I Kishan,10276,IND,WK,83.71,8.5,141,1,22,0,1
5,R Jadeja,587,IND,AR,76.24,9.0,34,1,93,1,1
6,D Shanaka,8422,SL,AR,68.26,9.0,32,1,77,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,0,0,1
8,B Kumar,1726,IND,BOWL,63.78,8.5,76,1,37,1,1
9,D Chameera,8393,SL,BOWL,72.85,9.0,30,1,31,1,1


In [13]:
match_2_points = sum(fdf2.Points)
printmd('**Total Points for Match 2: '+ str(match_2_points)+'**', 'blue')

<span style='color:blue'>**Total Points for Match 2: 522**</span>

In [15]:
dfMatch3 = pd.read_csv("ind_sl_t20i_2nd_2022_02_26.csv")
dfMatch3 = dfMatch3.join(pd.get_dummies(dfMatch3[['Role', 'Team']]))
dfMatch3.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,False,True,False,False,True,False
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,False,True,False,False,False,True
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,True,False,False,False,True,False
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,True,False,False,False,False,True
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,False,False,False,True,True,False


In [16]:

fdf3 = createFanstasyTeam(dfMatch3)

In [17]:
fdf3[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status
0,S Iyer,9428,IND,BAT,87.64,8.5,84,1,1
1,P Nissanka,13682,SL,BAT,66.77,9.5,2,1,1
2,R Sharma,576,IND,BAT,87.27,10.5,66,1,1
3,C Asalanka,10934,SL,BAT,77.97,8.5,70,1,1
4,I Kishan,10276,IND,WK,83.71,8.5,141,1,1
5,R Jadeja,587,IND,AR,76.24,9.0,34,1,1
6,D Shanaka,8422,SL,AR,68.26,9.0,32,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,1
8,B Kumar,1726,IND,BOWL,63.78,8.5,76,1,1
9,D Chameera,8393,SL,BOWL,72.85,9.0,30,1,1


In [18]:
fdf3[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'Points', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,Points,pickup_status
0,S Iyer,9428,IND,BAT,87.64,8.5,84,1,112,1
1,P Nissanka,13682,SL,BAT,66.77,9.5,2,1,100,1
2,R Sharma,576,IND,BAT,87.27,10.5,66,1,13,1
3,C Asalanka,10934,SL,BAT,77.97,8.5,70,1,6,1
4,I Kishan,10276,IND,WK,83.71,8.5,141,1,22,1
5,R Jadeja,587,IND,AR,76.24,9.0,34,1,93,1
6,D Shanaka,8422,SL,AR,68.26,9.0,32,1,77,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,0,1
8,B Kumar,1726,IND,BOWL,63.78,8.5,76,1,37,1
9,D Chameera,8393,SL,BOWL,72.85,9.0,30,1,31,1


In [19]:
match_3_points = sum(fdf3.Points)
printmd('**Total Points for Match 3: '+ str(match_3_points)+'**', 'blue')

<span style='color:blue'>**Total Points for Match 3: 522**</span>

In [20]:
mean_points = np.mean((match_1_points,match_2_points,match_3_points))
printmd('**Average Points for IND vs SL Series Per Match: '+ str(mean_points)+'**', 'green')

<span style='color:green'>**Average Points for IND vs SL Series Per Match: 495.0**</span>